## Displaying two networks in one using line style annotations

The following script sequence loads two gene regulatory network files
into a single network display
and adds line style annotations and colors which differentiate edges only occurring in
network1, edges occurring only in network2, and edges occurring in both
networks.

It should be possible to extend this approach for 3 or more network
combinations using more colors and line patterns.

First we add a utility for reading in the data from the files as a list
of dictionaries.

### Reading the data

In [ ]:
def load_dictionaries_from_file(f, limit=None, cutoff=None):
    """
    Load tab separated file into dictionary mapping edge --> dict
    """
    headers = f.readline().split()
    result = {}
    while True:
        line = f.readline()
        if not line:
            break
        values = line.split()
        dictionary = {h: v for (h, v) in zip(headers, values)}
        beta = float(dictionary['beta.sign.sum'])
        dictionary["beta"] = beta
        # ignore if beta below cutoff
        if abs(beta) < cutoff:
            continue
        edge = (dictionary["regulator"], dictionary["target"])
        result[edge] = dictionary
        if limit and len(result) >= limit:
            break
    return result

def load_file(filename, limit=None):
    return load_dictionaries_from_file(open(filename), limit=limit)

# simple test case usage:
import StringIO
buffer = """
regulator	target	beta.sign.sum	beta.non.zero	var.exp.median	combined_confidences	prior
BSU24520	BSU30750	-20.0	20.0	0.99724170989978111	0.99999556004762358	-1.0
BSU22440	BSU30200	-20.0	20.0	0.99649123580861043	0.99998929857632357	-1.0
""".strip()
f = StringIO.StringIO(buffer)
example_dicts = load_dictionaries_from_file(f, limit=1)
print example_dicts

## Adding annotations
Above illustrates how each line of the file read becomes a dictionary in a list of dictionsries.
We extend these dictionaries to add line styles using the special attributes `stroke`
and `stroke-dasharray` in the function `add_line_styles`:

In [ ]:
def add_line_styles(edge_to_dict, negative_color, positive_color, dasharray):
    "add line styles and colors for edges in dictionary sequence."
    for dictionary in edge_to_dict.values():
        beta = dictionary["beta"]
        dictionary["stroke-dasharray"] = dasharray
        if beta < 0:
            dictionary['stroke'] = negative_color
        else:
            dictionary['stroke'] = positive_color
    return edge_to_dict

print add_line_styles(example_dicts, "magenta", "cyan", "2,2,3")

## Reading the files

Using these functions read in the two networks, using dotted lines in cyan/magenta for the first network
and dash-dotted lines in blue/orange for the second network.

In [ ]:
# load network files and add annotations
dicts1 = load_file('./network1.tsv', limit=4000)
dicts1 = add_line_styles(dicts1, "magenta", "cyan", "1,1")
dicts2 = load_file('./network2.tsv', limit=4000)
dicts2 = add_line_styles(dicts2, "orange", "blue", "3,3,1,1") #edge_to_dict, negative_color, positive_color, dasharray

In [ ]:
#?add_line_styles()

## Loading the graph into a network display

Load the network data into a graph structure and change annotations
for shared edges.  Here shared edges will have solid lines in green/red.

In [ ]:
from jp_gene_viz import dGraph
graph = dGraph.WGraph()
for edge in dicts1:
    dictionary = dicts1[edge]
    if edge in dicts2:
        # change annotations for shared edges
        del dictionary["stroke-dasharray"]
        beta = dictionary["beta"]
        if beta < 0:
            dictionary['stroke'] = "red"  # negative_color
        else:
            dictionary['stroke'] = "green"  # positive_color
    (source, dest) = edge
    graph.add_edge(source, dest, dictionary["beta"], dictionary)
for edge in dicts2:
    if edge not in dicts1:
        dictionary = dicts2[edge]
        (source, dest) = edge
        graph.add_edge(source, dest, dictionary["beta"], dictionary)

Load the annotated graph into a network display and display the network widget.

In [ ]:
from jp_gene_viz import dNetwork, dLayout
dNetwork.load_javascript_support()

In [ ]:
# Load the graph into a network display
network = dNetwork.NetworkDisplay()
#positions = dLayout.group_layout(graph)[0]  # step around a bug here...
positions = dLayout.spoke_layout.spoke_layout(graph)[0]
network.load_data(graph, positions)

In [ ]:
network.show()

In [ ]:
print "total edges", len(set(dicts1).union(set(dicts2)))

In [ ]:
print "shared edges", len(set(dicts1) & set(dicts2))

Focus on "bsu257*" and neighbors.

In [ ]:
network.pattern_text.value = "ylaC"
network.match_click()
network.labels_button.value = True
network.expand_click()
network.layout_click()

In [ ]:
network.labels_button.value